### Codigo para la propagación de onda elástica 2D PSV en un medio anisotrópico VTI mediante el esquema velocidad - esfuerzo y rejilla escalonada aplicado a un VSP zero offset ILM13 (usado en tesis)


In [17]:
# Import Libraries 
# ----------------------------------------------
%matplotlib notebook
import numpy as np
import numpy.matlib as npmat
import matplotlib.pyplot as plt

In [18]:
def d(comp,inpu,dire):
    if(comp=='x'): idim = 0
    elif(comp=='z'): idim = 1
    
    if(dire == '+'):
        return (9/8*(np.roll(inpu, -1, axis = idim)-inpu)
                -1/24*(np.roll(inpu, -2, axis = idim)-np.roll(inpu, 1, axis = idim)))/dx
    elif(dire == '-'):
        return (9/8*(inpu - np.roll(inpu, 1, axis = idim))
                -1/24*(np.roll(inpu, -1, axis = idim)-np.roll(inpu, 2, axis = idim)))/dx

In [19]:
# Input Parameter
# ----------------------------------------------

# Discretization
c1=20           # Number of grid points per dominant wavelength
c2=0.6          # CFL-Number for 2D CFL< 0.606
nx=34          # number of grid points in x-direction
nz=1058          # number of grid points in z-direction
T=1.5
idisp = 100# display frequency

# The grid number of pml
pml = 150                              
m = nx + 2*pml
n = nz + pml + 1
 
# Source Signal
f0= 20      # Center frequency Ricker-wavelet
q0= 1      # Maximum amplitude Ricker-Wavelet
t0= 1.5/f0
xscr = pml+33;  zscr = 1#pml+6 # Source position (in grid points)



In [20]:
# Velocity and density
Vpp = np.loadtxt('Vp_ILM13.txt') # P-wave velocity in m/s
Vpp=Vpp.transpose()
Vss = np.loadtxt('Vs_ILM13.txt') # S-wave velocity in m/s
Vss=Vss.transpose()
den= np.loadtxt('rhob_ILM13.txt') # Density in Kg/m^3
den=den.transpose()
eps= np.loadtxt('epsilon_ILM13.txt') # Epsilon
eps=eps.transpose()
delt= np.loadtxt('delta_ILM13.txt') # Epsilon
delt=delt.transpose()

In [21]:
# Plotting velocity and density model
plt.ion()
fig1 = plt.figure()
#plt.title('P-wave Velocity Model (m/s)')
plt.xlabel('x (m)')
plt.ylabel('z (m)')
im = plt.imshow(Vpp.transpose(),vmin=Vpp.min(), vmax=Vpp.max(), animated=True,
                interpolation="nearest", cmap=plt.cm.RdBu)
fig1.colorbar(im)

fig2 = plt.figure()
plt.title('S-wave Velocity Model (m/s)')
plt.xlabel('x-grid')
plt.ylabel('z-grid')
im = plt.imshow(Vss.transpose(),vmin=Vss.min(), vmax=Vss.max(), animated=True,
                interpolation="nearest", cmap=plt.cm.RdBu)
fig2.colorbar(im)

fig3 = plt.figure()
plt.title('Density Model (Kg/$m^3$)')
plt.xlabel('x-grid')
plt.ylabel('z-grid')
im = plt.imshow(den.transpose(),vmin=den.min(), vmax=den.max(), animated=True,
                interpolation="nearest", cmap=plt.cm.RdBu)
fig3.colorbar(im)

plt.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
# PML extra edge

Vp=np.zeros((m,n))                        # medium velocity of P wave,m/s,zero fill
Vp[pml:pml+nx,1:nz+1] = Vpp
Vp[pml:pml+nx,0] = Vpp[:,0]
Vp[pml:pml+nx,nz+1:n] = np.transpose(npmat.repmat(Vpp[:,nz-1],pml,1))
Vp[0:pml,:] = npmat.repmat(Vp[pml,:],pml, 1)
Vp[pml+nx:2*pml+nx,:] = npmat.repmat(Vp[pml+nx-1,:],pml, 1)
del Vpp

Vs=np.zeros((m,n))                        # medium velocity of S wave,m/s,zero fill
Vs[pml:pml+nx,1:nz+1] = Vss
Vs[pml:pml+nx,0] = Vss[:,0]
Vs[pml:pml+nx,nz+1:n] = np.transpose(npmat.repmat(Vss[:,nz-1],pml,1))
Vs[0:pml,:] = npmat.repmat(Vs[pml,:],pml, 1)
Vs[pml+nx:2*pml+nx,:] = npmat.repmat(Vs[pml+nx-1,:],pml, 1)
del Vss

rho=np.zeros((m,n))                        # medium density,kg/m**3,zero fill
rho[pml:pml+nx,1:nz+1] = den
rho[pml:pml+nx,0] = den[:,0]
rho[pml:pml+nx,nz+1:n] = np.transpose(npmat.repmat(den[:,nz-1],pml,1))
rho[0:pml,:] = npmat.repmat(rho[pml,:],pml, 1)
rho[pml+nx:2*pml+nx,:] = npmat.repmat(rho[pml+nx-1,:],pml, 1)
del den

epsilon=np.zeros((m,n))                        # medium epsilon,zero fill
epsilon[pml:pml+nx,1:nz+1] = eps
epsilon[pml:pml+nx,0] = eps[:,0]
epsilon[pml:pml+nx,nz+1:n] = np.transpose(npmat.repmat(eps[:,nz-1],pml,1))
epsilon[0:pml,:] = npmat.repmat(epsilon[pml,:],pml, 1)
epsilon[pml+nx:2*pml+nx,:] = npmat.repmat(epsilon[pml+nx-1,:],pml, 1)
del eps

delta=np.zeros((m,n))                        # medium delta,zero fill
delta[pml:pml+nx,1:nz+1] = delt
delta[pml:pml+nx,0] = delt[:,0]
delta[pml:pml+nx,nz+1:n] = np.transpose(npmat.repmat(delt[:,nz-1],pml,1))
delta[0:pml,:] = npmat.repmat(delta[pml,:],pml, 1)
delta[pml+nx:2*pml+nx,:] = npmat.repmat(delta[pml+nx-1,:],pml, 1)
del delt

# Calculate Thomsen Parameters
C33 = Vp**2*rho
C44 = Vs**2*rho
C11 = C33*(2*epsilon + 1)
C13 = np.sqrt(2*C33*(C33-C44)*delta + (C33-C44)**2) - C44
del Vs

plt.ion()
fig1 = plt.figure()
#plt.title('P-wave Velocity Model (m/s)')
plt.xlabel('x (m)')
plt.ylabel('z (m)')
im = plt.imshow(Vp.transpose(),vmin=Vp.min(), vmax=Vp.max(), animated=True,
                interpolation="nearest", cmap=plt.cm.RdBu,extent=[0,(m)*1.5,(n)*1.5,0],aspect = 'equal')

<IPython.core.display.Javascript object>

In [23]:
## Preparation 

# Init wavefields
vx=np.zeros((m,n))
vz=np.zeros((m,n))
sxx=np.zeros((m,n))
szz=np.zeros((m,n))
sxz=np.zeros((m,n))
vx1=np.zeros((m,n))
vx2=np.zeros((m,n))
vz1=np.zeros((m,n))
vz2=np.zeros((m,n))
sxx1=np.zeros((m,n))
sxx2=np.zeros((m,n))
szz1=np.zeros((m,n))
szz2=np.zeros((m,n))
sxz1=np.zeros((m,n))
sxz2=np.zeros((m,n))

#vol   = np.zeros([m,n])
#curl  = np.zeros([m,n])

cmin=Vp.min()  # Lowest P-wave velocity
cmax=Vp.max()  # Highest P-wave velocity
fmax=2*f0              # Maximum frequency
#dx=cmin/(fmax*c1)      # Spatial discretization (in m)
dx=0.999744
dz=dx 
#dt=dx/(cmax)*c2        # Temporal discretization (in s)
dt=0.00004
lampda_min=cmin/fmax   # Smallest wavelength
print(dx,dz,dt)
del Vp
ond = np.loadtxt('ond_vsp.txt') 
# Create space and time vector
x=np.arange(0,dx*nx-dx,dx) # Space vector
z=np.arange(0,(dz*nz-dz),dz) # Space vector
#t=np.arange(0,(T-dt),dt)    # Time vector
t=ond[:,0]
nt=len(t)       # Number of time steps

# Shot gathers
seismo_vx=np.zeros((nt,n))
seismo_vz=np.zeros((nt,n))

0.999744 0.999744 4e-05


MemoryError: 

In [ ]:
# Source signal - Ricker-wavelet

# Initialise source
#tau=np.pi*f0*(t-t0);
#s=q0*(1-2*tau**2)*np.exp(-tau**2);
s=ond[:,1]
# Plotting source signal
plt.ion()
fig4 = plt.figure(figsize=(9,4.5))
plt.plot(t-t0,s)
plt.title('Source signal Ricker-Wavelet')
plt.xlabel('Time in s')
plt.ylabel('Amplitude')
plt.show() 

In [ ]:
#**********************attemuatiom(pnl)************************
R=4.76354e-07          # theoretical reflectiom coefficiemt
ddx=np.zeros((m,n)) 
ddz=np.zeros((m,n))
plx=pml*dx;
plz=pml*dz;
for i in range(0,m):
     for k in range(0,n):
         # zome1
         if i>=0 and i<=pml-1 and k>=0 and k<=pml-1:
             z=pml-k;x=pml-i;
             #ddz[i,k]=-np.log(R)*((3*cmax)/(2*plx))*(z**2/plx**2)
             ddx[i,k]=-np.log(R)*((3*cmax)/(2*plx))*(x**2/plx**2)
         elif i>=0 and i<=pml-1 and k>n-pml-1 and k<=n:
             z=k-(n-pml);x=pml-i;
             ddz[i,k]=-np.log(R)*((3*cmax)/(2*plx))*(z**2/plx**2)
             ddx[i,k]=-np.log(R)*((3*cmax)/(2*plx))*(x**2/plx**2)
         elif i>m-pml-1 and i<=m and k>=0 and k<=pml-1:
             z=pml-k;x=i-(m-pml);
             #ddz[i,k]=-np.log(R)*((3*cmax)/(2*plx))*(z**2/plx**2)
             ddx[i,k]=-np.log(R)*((3*cmax)/(2*plx))*(x**2/plx**2)
         elif i>m-pml-1 and i<=m and k>n-pml-1 and k<=n:
             z=k-(n-pml);x=i-(m-pml);
             ddz[i,k]=-np.log(R)*((3*cmax)/(2*plx))*(z**2/plx**2)
             ddx[i,k]=-np.log(R)*((3*cmax)/(2*plx))*(x**2/plx**2)
  
         # xone2
         elif i<=pml-1 and k>pml-1 and k<n-pml:
             z=0;x=pml-i;
             ddz[i,k]=0;
             ddx[i,k]=-np.log(R)*((3*cmax)/(2*plx))*(x**2/plx**2)
         elif  i>m-pml-1 and i<=m and k>pml-1 and k<=n-pml-1:
             z=0;x=i-(m-pml);
             ddz[i,k]=0;
             ddx[i,k]=-np.log(R)*((3*cmax)/(2*plx))*(x**2/plx**2)
         # xome3
         elif i>pml-1 and i<=m-pml-1 and k<=pml-1:
             z=pml-k;x=0;
             #ddz[i,k]=-np.log(R)*((3*cmax)/(2*plx))*(z**2/plx**2)
             ddx[i,k]=0;
         elif i>pml-1 and i<=m-pml-1 and k>n-pml-1 and k<=n:
             z=k-(n-pml);x=0;
             ddz[i,k]=-np.log(R)*((3*cmax)/(2*plx))*(z**2/plx**2)
             ddx[i,k]=0;
             
#ddx=np.transpose(ddx)
#ddz=np.transpose(ddz)
plt.ion()
fig1 = plt.figure()
#plt.title('P-wave Velocity Model (m/s)')
plt.xlabel('x (m)')
plt.ylabel('z (m)')
im = plt.imshow(ddx,vmin=ddz.min(), vmax=ddx.max(), animated=True,
                interpolation="nearest", cmap=plt.cm.RdBu,aspect = 'equal')

In [ ]:
# Plot preparation
# Initialize animated plot
plt.ion()
fig5 = plt.figure(figsize=(9,6))
f1 = fig5.add_subplot(2, 2, 1)
f2 = fig5.add_subplot(2, 2, 2)
f3 = fig5.add_subplot(2, 2, 3)
f4 = fig5.add_subplot(2, 2, 4)
f1.set_title('sxx')
f2.set_title('szz')
f3.set_title('vx')
f4.set_title('vz')

curlp = f1.imshow(sxx[pml:m-pml,1:n-pml].transpose(), interpolation='nearest', animated=True,
                   vmin=-2.5e2, vmax=2.5e2, cmap=plt.cm.RdBu,extent=[0,(m-2*pml)*dx,(n-2*pml)*dz,0],aspect = 'auto')
divp = f2.imshow(szz[pml:m-pml,1:n-pml].transpose(), interpolation='nearest', animated=True,
                   vmin=-2.5e2, vmax=2.5e2, cmap=plt.cm.RdBu,extent=[0,(m-2*pml)*dx,(n-2*pml)*dz,0],aspect = 'auto')
vxp = f3.imshow(vx[pml:m-pml,1:n-pml].transpose(), interpolation='nearest', animated=True,
                   vmin=-2.5e2, vmax=2.5e2, cmap=plt.cm.RdBu,extent=[0,(m-2*pml)*dx,(n-2*pml)*dz,0],aspect = 'auto')
vzp = f4.imshow(vz[pml:m-pml,1:n-pml].transpose(), interpolation='nearest', animated=True,
                   vmin=-2.5e2, vmax=2.5e2, cmap=plt.cm.RdBu,extent=[0,(m-2*pml)*dx,(n-2*pml)*dz,0],aspect = 'auto')
plt.show()

In [ ]:
for tt in range(1,nt):

    # Inject source wavelet (explosion)
       
    sxx[xscr,zscr]=sxx[xscr,zscr]+s[tt]   
    szz[xscr,zscr]=szz[xscr,zscr]+s[tt]
    
    # Free surface
    vx[:,0]= vx[:,1] + dz*d('x',vz[:,1],'-')
    vz[:,0]= vz[:,1] + (C33[:,1]/C13[:,1])*d('x',vx[:,1],'+')
               
    # Update velocity
    vx1=((1-0.5*dt*ddx)*vx1+(dt/rho)*d('x',sxx,'-'))/(1+0.5*dt*ddx)  
    vx2=((1-0.5*dt*ddz)*vx2+(dt/rho)*d('z',sxz,'-'))/(1+0.5*dt*ddz)
    vx = vx1+vx2
    
    vz1=((1-0.5*dt*ddx)*vz1+(dt/rho)*d('x',sxz,'+'))/(1+0.5*dt*ddx)
    vz2=((1-0.5*dt*ddz)*vz2+(dt/rho)*d('z',szz,'+'))/(1+0.5*dt*ddz)
    vz = vz1+vz2         
                       
    # Update pressure
    sxx1=((1-0.5*dt*ddx)*sxx1+C11*dt*d('x',vx,'+'))/(1+0.5*dt*ddx)
    sxx2=((1-0.5*dt*ddz)*sxx2+C13*dt*d('z',vz,'-'))/(1+0.5*dt*ddz)
    sxx = sxx1 + sxx2    

    szz1=((1-0.5*dt*ddx)*szz1+C13*dt*d('x',vx,'+'))/(1+0.5*dt*ddx)
    szz2=((1-0.5*dt*ddz)*szz2+C33*dt*d('z',vz,'-'))/(1+0.5*dt*ddz) 
    szz = szz1+szz2 

    sxz1=((1-0.5*dt*ddx)*sxz1+C44*dt*d('x',vz,'-'))/(1+0.5*dt*ddx)
    sxz2=((1-0.5*dt*ddz)*sxz2+C44*dt*d('z',vx,'+'))/(1+0.5*dt*ddz)
    sxz = sxz1+sxz2 
    
    seismo_vx[tt,0:n]=vx[pml,0:n]
    seismo_vz[tt,0:n]=vz[pml,0:n]
    
    # Plot every time step (nt)
    if (tt % idisp) == 0:
        curlp.set_data(sxx[pml:m-pml,1:n-pml].transpose())
        f1.set_title("Time Step (nt) = %i" % (tt))
        divp.set_data(szz[pml:m-pml,1:n-pml].transpose())
        vxp.set_data(vx[pml:m-pml,1:n-pml].transpose())
        vzp.set_data(vz[pml:m-pml,1:n-pml].transpose())
        plt.gcf().canvas.draw()
        


In [ ]:
# Plot Shot gathers 5 pts below free surface
    
fac=1e-3
ymx=abs(seismo_vx).max()
clim=np.array([-fac*ymx, fac*ymx])
plt.ion()
fig2 = plt.figure()
plt.title('Common Shot Gather (horizontal component phone)')
plt.xlabel('x-grid')
plt.ylabel('nt')
im = plt.imshow(seismo_vx,vmin=clim.min(), vmax=clim.max(), animated=True,
                interpolation="nearest", cmap=plt.cm.RdBu,aspect = 'auto')
fig2.colorbar(im)
plt.show()
#plt.savefig("DFDT_het_2D_vx.png", bbox_inches='tight')

fac=1e-3
ymx=abs(seismo_vz).max()
clim=np.array([-fac*ymx, fac*ymx])
plt.ion()
fig3 = plt.figure()
plt.title('Common Shot Gather (vertical component phone)')
plt.xlabel('x-grid')
plt.ylabel('nt')
im = plt.imshow(seismo_vz,vmin=clim.min(), vmax=clim.max(), animated=True,
                interpolation="nearest", cmap=plt.cm.RdBu,aspect = 'auto')
fig3.colorbar(im)
plt.show()
#plt.savefig("DFDT_het_2D_vz.png", bbox_inches='tight')

In [ ]:
data_vx=np.zeros((1500,1058+pml+1))
for i in range(0,1499): 
    data_vx[i,:]=seismo_vx[i*25,:]
np.savetxt('seis_vx_VTI_ILM13.txt',data_vx)

data_vz=np.zeros((1500,1058+pml+1))
for i in range(0,1499): 
    data_vz[i,:]=seismo_vz[i*25,:]
np.savetxt('seis_vz_VTI.txt_ILM13',data_vz)        